---
title: Part 2. Introduction to Actinia
description: Learn how to use actinia to create a cloud-based processing environment for GRASS GIS.
format: html
author: Corey T. White
date: '2024-09-10'
keep-ipynb: true
toc: true
toc-depth: 4
image:  images/webinar_title.webp
categories: [geospatial, GRASS v8.5, jupyter, lidar, STAC]
page-layout: full
title-block-banner: true
---

## REST API for GRASS GIS

- [GRASS GIS](https://grass.osgeo.org/) is a Geospatial Processing Engine
- Open Source (GPL v2)
- Developed by International and Multi-institutional groups and individuals (GRASS Development Team)
- Member of the Open Source Geospatial Foundatispace
- Recieved Open Source Security Foundation (OpenSSF) Best Practices Badge - 2024

```{mermaid}
flowchart LR
  A[Client] -->|Http Request| B[Actinia REST API]
  B <--> D[GRASS GIS Processing Node]
  B -->|Http Response| A
```


## Import Python Libraries

In [2]:
import os
import subprocess
from pprint import pprint
import sys
import json
import time

import requests
from requests.auth import HTTPBasicAuth

## Setup Environment

In [27]:
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
GISBASE = gisbase
ACTINIA_USER = 'actinia-gdi'
ACTINIA_PASSWORD = 'actinia-gdi'
AUTH = 'actinia-gdi:actinia-gdi'
ACTINIA_URL = 'http://localhost:8088'
ACTINIA_URL = 'https://openplains.app/actinia'
ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_BASEURL = 'https://openplains.app/actinia'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth(ACTINIA_USER, ACTINIA_PASSWORD)

## actinia REST API

[actinia](https://github.com/actinia-org)

### Tools

- [actinia-python-client](https://github.com/actinia-org/actinia-python-client)
- [actinia_openapi_python_client](https://github.com/OpenPlainsInc/actinia_openapi_python_client)

In [ ]:
!pip3 install actinia-python-client
!pip3 install git+https://github.com/openplainsinc/actinia_openapi_python_client.git@v0.0.4

## Acinia Basics

In [30]:
url = f"{ACTINIA_URL}/version"
response = requests.get( url)
pprint(response.json())
# pprint(response.json().get('version'))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### GRASS Projects

Let's start by requesting a list of GRASS projects from actinia.

In [18]:
url = f"{ACTINIA_URL}/locations"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('locations'))


['nc_spm_08',
 'test_location',
 'Durham',
 'myloc',
 'sarahs_durham',
 'Test_Location']


Now we will get extra information about a specific project.

In [17]:
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/info"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

{'projection': 'PROJCRS["NAD83(HARN) / North '
               'Carolina",BASEGEOGCRS["NAD83(HARN)",DATUM["NAD83 (High '
               'Accuracy Reference Network)",ELLIPSOID["GRS '
               '1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4152]],CONVERSION["SPCS83 '
               'North Carolina zone (meters)",METHOD["Lambert Conic Conformal '
               '(2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false '
               'origin",33.75,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude '
               'of false '
               'origin",-79,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude '
               'of 1st standard '
               'parallel",36.1666666666667,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude '
               'of 2nd standard '
               'parallel",34.3333333333333,ANGLEUNIT["degree",0.01745

### Mapsets

We will now get a list of mapsets for a specific project.

In [22]:
# mapset = 'nc_spm_08'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

['PERMANENT']


Now let's get extra information about a specific mapset `PERMANENT`.

In [16]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/info"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

{'projection': 'PROJCRS["NAD83(HARN) / North '
               'Carolina",BASEGEOGCRS["NAD83(HARN)",DATUM["NAD83 (High '
               'Accuracy Reference Network)",ELLIPSOID["GRS '
               '1980",6378137,298.257222101,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4152]],CONVERSION["SPCS83 '
               'North Carolina zone (meters)",METHOD["Lambert Conic Conformal '
               '(2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false '
               'origin",33.75,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude '
               'of false '
               'origin",-79,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude '
               'of 1st standard '
               'parallel",36.1666666666667,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude '
               'of 2nd standard '
               'parallel",34.3333333333333,ANGLEUNIT["degree",0.01745

### Layers

#### Raster Layers

Let's see all of the mapsets raster layers.

In [21]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/raster_layers"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

[]


#### Vector Layers

In [20]:
mapset_id = 'PERMANENT'
location_id = 'nc_spm_08'
url = f"{ACTINIA_URL}/locations/{location_id}/mapsets/{mapset_id}/vector_layers"

payload = {}
headers = {}

response = requests.get( url, auth=ACTINIA_AUTH, headers=headers)

pprint(response.json().get('process_results'))

[]
